In [90]:
import pandas as pd

# 1. LOAD DATA

In [91]:
files = {
    "bao_chau": "../data/raw/bao_chau_data.csv",
    "duy_thai": "../data/raw/duy_thai_data.csv",
    "minh_huy": "../data/raw/minh_huy_data.csv",
    "quoc_trung": "../data/raw/quoc_trung_data.csv"
}

dfs = []
for name,path in files.items():
    df = pd.read_csv(
    path,
    sep=None,
    engine="python"
)

    df.columns = df.columns.str.lower().str.strip()
    df = df.rename(columns={
        'product name': 'product_name',
        'describe': 'description',
        'brand': 'brand',
        'category': 'category'
    })

    expected_cols = ["product_name", "description", "brand", "category"]
    df = df[[c for c in expected_cols if c in df.columns]]

    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6243 entries, 0 to 6242
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   description   6240 non-null   object
 1   brand         6057 non-null   object
 2   category      6243 non-null   object
 3   product_name  901 non-null    object
dtypes: object(4)
memory usage: 195.2+ KB


# 2. XỬ LÝ MISSING VALUES

In [92]:
df.isnull().sum()

description        3
brand            186
category           0
product_name    5342
dtype: int64

In [93]:
num_null_brand = df["brand"].isna().sum()
ratio = num_null_brand / len(df) * 100
print(f"Tỷ lệ thiếu brand: {ratio:.2f}%")

Tỷ lệ thiếu brand: 2.98%


Nhận xét về dữ liệu thiếu
- Không có giá trị thiếu ở `product_name` và `description`
- Cột `brand` có 187 giá trị bị thiếu, chiếm tỷ lệ 5.19%
- Cột `category` có 1 giá trị bị thiếu

Hướng giải quyết:
- Với brand: điền giá trị mặc định `'Unknown'`
- Với `category`: xóa bỏ, do chỉ có 1 dòng

In [94]:
df = df.dropna(subset=['category'])
df["brand"] = df["brand"].fillna("Unknown")

In [95]:
#KIỂM TRA LẠI
print("Số lượng missing values sau khi xử lý:")
print(df.isnull().sum())

print(f"\nKích thước dữ liệu hiện tại: {df.shape}")

Số lượng missing values sau khi xử lý:
description        3
brand              0
category           0
product_name    5342
dtype: int64

Kích thước dữ liệu hiện tại: (6243, 4)


# 3. XỬ LÝ DUPLICATES

In [96]:
dup_mask = df.duplicated(subset=["product_name", "brand", "category"], keep=False)

dup_by_category = (
    df[dup_mask]
    .groupby("category")
    .size()
    .reset_index(name="num_duplicates")
    .sort_values("num_duplicates", ascending=False)
)

print(f"Tổng số dòng trùng: {dup_mask.sum()}")
print(f"Tỉ lệ dòng trùng: {dup_mask.sum() / len(df) * 100:.2f}%")
dup_by_category

Tổng số dòng trùng: 4874
Tỉ lệ dòng trùng: 78.07%


,category,num_duplicates
0,Giày dép,585
6,Thời trang Nam,581
7,Thời trang nữ,579
8,Điện thoại & Phụ kiện,571
9,Đồ chơi trẻ em,529
10,Đồ gia dụng,521
4,Sách & Văn phòng phẩm,506
2,Mỹ phẩm & Làm đẹp,503
1,Laptop & Máy tính,479
3,Nhà cửa & Đời sống,16


Hướng giải quyết:
- Do tỷ lệ trùng lặp nhỏ, nên lựa chọn loại bỏ các dòng trùng lặp để dữ liệu sạch hơn

In [97]:
df_cleaned = df.drop_duplicates(
    subset=['product_name', 'brand', 'category'],
    keep='first'
).copy()

print(f"Tổng số dòng sau khi loại duplicates: {len(df_cleaned)}")
print(f"Đã loại bỏ: {len(df) - len(df_cleaned)} dòng")

Tổng số dòng sau khi loại duplicates: 1943
Đã loại bỏ: 4300 dòng


# 4. CHUẨN HÓA TEXT TIẾNG VIỆT

In [98]:
import re
from bs4 import BeautifulSoup

In [99]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # 1. Xử lý HTML tags
    try:
        soup = BeautifulSoup(text, "html.parser")
        for data in soup(['style', 'script', 'head', 'title', 'meta', '[document]']):
            data.decompose()
        text = soup.get_text(" ")
    except:
        pass    
    # 2. Chuyển thành chữ thường
    text = text.lower()
    # 3. Loại bỏ URL và email
    text = re.sub(r"http\S+|www\S+|\S+@\S+", " ", text)
    # 4. Loại bỏ các ký tự xuống dòng, tab đặc biệt (\n, \t, \r, \v)
    text = re.sub(r"[\n\t\r\v]+", " ", text)
    # 5. Chuẩn hóa các từ nối dài
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    # 6. Loại bỏ ký tự đặc biệt (giữ tiếng Việt + số)
    text = re.sub(
        r"[^0-9a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\s]",
        " ",
        text
    )
    # 7. Loại bỏ số điện thoại, mã số...
    text = re.sub(r'\d{10,}', '', text)
    # 8. Loại bỏ khoảng trắng thừa
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [100]:
for col in ["product_name", "description", "brand"]:
    df_cleaned.loc[:, col] = df_cleaned[col].apply(clean_text)
df_cleaned.head()

/var/folders/sp/zbn29lqn02g4m4zhf168jm5r0000gn/T/ipykernel_91119/4014682164.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,description,brand,category,product_name
0,thông tin người mặc size s s ả n ph ẩ m vòng e...,oem,Thời trang nữ,
2,shop chuyên cung cấp quần âu áo sơ mi áo kiểu ...,arctic hunter,Thời trang nữ,
9,shop chuyên cung cấp quần âu áo sơ mi áo kiểu ...,haint boutique,Thời trang nữ,
13,váy nữ cổ phối tơ voan màu đen 23v037 pi style...,pi home of beauty,Thời trang nữ,
16,chất liệu cotton mềm mại thoáng khí thấm hút m...,hity,Thời trang nữ,


Nhận xét:
- Cột `Description` còn nhiều dòng bắt đầu bằng các cụm từ không có ý nghĩa, lặp lại nhiều lần: "thông tin sản phẩm", "giới thiệu sản phẩm", "chi tiết sản phẩm"...
- Lựa chọn loại bỏ để thông tin giá trị hơn

In [101]:
def remove_boilerplate(text):
    if not isinstance(text, str) or len(text) == 0:
        return ""
    patterns = [
        r"^mô tả sản phẩm",
        r"^thông tin sản phẩm", 
        r"^chi tiết sản phẩm",
        r"^thông số kỹ thuật",
        r"^giới thiệu sản phẩm"
    ]
    combined_pattern = "|".join(patterns)
    text = re.sub(combined_pattern, "", text).strip()
    return text

# Áp dụng CHỈ cho cột description
df_cleaned['description'] = df_cleaned['description'].apply(remove_boilerplate)

In [102]:
#KIỂM TRA LẠI SAU XỬ LÝ CÓ CHUỖI RỖNG KHÔNG
def check_empty_strings(df):
    for col in df.columns:
        if df[col].dtype == "object":
            num_empty = (df[col].str.strip() == "").sum()
            print(f"Số chuỗi rỗng trong cột '{col}': {num_empty}")
check_empty_strings(df_cleaned)

Số chuỗi rỗng trong cột 'description': 2
Số chuỗi rỗng trong cột 'brand': 2
Số chuỗi rỗng trong cột 'category': 0
Số chuỗi rỗng trong cột 'product_name': 1052


In [103]:
#THAY THẾ CHUỖI RỖNG BẰNG 'unknown'
df_cleaned['brand'] = df_cleaned['brand'].replace('', 'unknown')

In [104]:
#KIỂM TRA CHẤT LƯỢNG DATA
df_cleaned['name_length'] = df_cleaned['product_name'].str.len()
df_cleaned['desc_length'] = df_cleaned['description'].str.len()
df_cleaned['name_word_count'] = df_cleaned['product_name'].str.split().str.len()
df_cleaned['desc_word_count'] = df_cleaned['description'].str.split().str.len()

print("Thống kê độ dài:")
print(f"Product name - Trung bình: {df_cleaned['name_length'].mean():.1f} ký tự")
print(f"Description - Trung bình: {df_cleaned['desc_length'].mean():.1f} ký tự")
print(f"Product name - Trung bình: {df_cleaned['name_word_count'].mean():.1f} từ")
print(f"Description - Trung bình: {df_cleaned['desc_word_count'].mean():.1f} từ")

Thống kê độ dài:
Product name - Trung bình: 19.2 ký tự
Description - Trung bình: 1000.8 ký tự
Product name - Trung bình: 3.7 từ
Description - Trung bình: 225.8 từ


Nhận xét:    
- Một số sản phẩm có tên quá ngắn hoặc mô tả quá ít từ, hay tên chỉ là số, không đủ thông tin.  
- Hướng xử lý: loại bỏ các sản phẩm không đạt tiêu chuẩn tối thiểu (tên ≥2 từ, mô tả ≥5 từ, tên không phải số) để đảm bảo dữ liệu sạch và chất lượng hơn.


In [105]:
#LOẠI BỎ CÁC SẢN PHẨM KHÔNG ĐỦ TIÊU CHUẨN
df_cleaned = df_cleaned[
    (df_cleaned['name_word_count'] >= 2) &  # Tên ít nhất 2 từ
    (df_cleaned['desc_word_count'] >= 5)  &   # Mô tả ít nhất 5 từ
    (df_cleaned['product_name'].str.isnumeric() == False)  # Tên không phải là số
].copy()
print(f"Số dòng sau khi loại bỏ sản phẩm không đủ tiêu chuẩn: {len(df_cleaned)}")
print(f"Số dòng đã loại bỏ: {len(df) - len(df_cleaned)}")

Số dòng sau khi loại bỏ sản phẩm không đủ tiêu chuẩn: 889
Số dòng đã loại bỏ: 5354


# 5. DỮ LIỆU ĐÃ LÀM SẠCH

In [106]:
columns_to_save = ['product_name', 'description', 'brand', 'category']
df_cleaned[columns_to_save].to_csv('../data/processed/data_cleaned.csv', index=False, encoding='utf-8')